In [7]:
# Load the dataset into PyTactician's visualizer.
from pytact import data_reader, graph_visualize_browse
import pathlib
from typing import Optional, List, DefaultDict
from pytact.data_reader import Node
from pytact.graph_api_capnp_cython import EdgeClassification
from pytact.graph_api_capnp_cython import Graph_Node_Label_Which
from collections import defaultdict
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import random
import numpy as np
from sklearn.metrics import classification_report
from typing import List

In [8]:


class TreeRNNCell(nn.Module):
    def __init__(self, hidden_size: int, edges_number: int):
        super(TreeRNNCell, self).__init__()
        # Initialize the Up matrix as a trainable parameter
        self.Up = nn.Parameter(torch.randn(edges_number, hidden_size, hidden_size))
        self.hidden_size = hidden_size

    def forward(self, h_, e, x: torch.Tensor) -> torch.Tensor:
        # Calculate the mean of transformed child hidden states
        child_agg = torch.mean(torch.stack([
            torch.mm(h, self.Up[edge_idx]) for h, edge_idx in zip(h_,e)
        ]), dim=0)
     
        # Apply non-linear transformation and sum with input tensor x
        return torch.tanh(child_agg + x)


class DecodeEmbedding(nn.Module):
    def __init__(self, hidden_size: int, edges_number: int):
        super(DecodeEmbedding, self).__init__()
        # Initialize the Ep matrix as a trainable parameter
        self.Ep = nn.Parameter(torch.randn(edges_number, hidden_size, hidden_size))
        # Initialize cp as a trainable parameter 
        self.cp = nn.Parameter(torch.zeros(1, hidden_size))

    def forward(self, h: torch.Tensor, edge_idx: int) -> torch.Tensor:
        # Ensure cp is broadcasted correctly over the batches
        transformed_h = torch.mm(h, self.Ep[edge_idx])
        # Add cp (broadcasted) to the result of the matrix multiplication
        # Note: If cp is intended to be a fixed bias, it should be initialized outside the parameter list
        return torch.sigmoid(transformed_h + self.cp)  # squeeze cp to match dimensions



class RNNEncoderDecoderClasifier(nn.Module): 
    def __init__(self, hidden_size: int, edges_number: int, nodes_number: int): 
        super(RNNEncoderDecoderClasifier, self).__init__()
        self.emb = nn.Embedding(nodes_number, hidden_size)
        self.dec = DecodeEmbedding(hidden_size, edges_number)
        self.enc = TreeRNNCell(hidden_size, edges_number)
        self.R = nn.Linear(hidden_size, nodes_number, bias=True) #output
      
    def encode_dag(self, dag):
        def encode_node(node): 
            if node.children and node.label.which.name != 'REL': 
                h = [] 
                e = [] 
                for edge_label, child in node.children: 
                    h.append(encode_node(child))
                    e.append(edge_label.value)
                return self.enc(h, e, self.emb(torch.tensor(node.label.which.value)))
            else: 
                return self.emb(torch.tensor(node.label.which.value)).unsqueeze(0)
        encoding = encode_node(dag)
        return encoding
    
    def decode_dag(self, dag, h, max_depth):
        decoded_graph = []
        def decode_node(node, h, depth, max_depth):
            logits = self.R(h)
            decoded_graph.append((torch.softmax(logits, dim=-1), depth))
            if depth < max_depth and node.children and not node.label.which.name == 'REL':
                for edge_label, child in node.children: 
                    h = self.dec(h, edge_label.value)
                    decode_node(child, h, depth+1, max_depth)

        decode_node(dag, h, 1, max_depth)
        return decoded_graph
    
    def forward(self, dag, max_depth):
        enc = self.encode_dag(dag)
        return self.decode_dag(dag, enc, max_depth)
    
class LabelGetter: 
    def __init__(self): 
        self.labels = []
    def get_labels(self, graph, max_depth):
        self.labels = []
        self.get_labels_helper(graph, 1, max_depth)
        return self.labels
    def get_labels_helper(self, graph, depth, max_depth):
        self.labels.append(graph.label.which.value)
        if graph.children and not graph.label.which.name == 'REL' and depth < max_depth: 
            for _, child in list(graph.children):
                self.get_labels_helper(child, depth+1, max_depth)
                
def get_file_size(reader, dataset_pointer): 
        pdl = dataset_pointer.lowlevel
        size = len(pdl.graph.nodes)
        return size

In [9]:
# Constants and configurations
DATASET_PATH = '../../../../v15-stdlib-coq8.11/dataset'
FILE_PATH = "coq-tactician-stdlib.8.11.dev/theories/Init/Logic.bin"
DATASET_PATH = pathlib.Path(DATASET_PATH)
FILE_PATH = pathlib.Path(FILE_PATH)

In [11]:
# Randomness
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

# Model Parameters 
NODES_NUMBER = 30
EMBEDDING_SIZE = 8
HIDDEN_SIZE = 16
EDGES_NUMBER = 50

# Model Introduction
model = RNNEncoderDecoderClasifier(HIDDEN_SIZE, EDGES_NUMBER, NODES_NUMBER)
lg = LabelGetter() #graph node_labels extractor

# Model Training Details
LEARNING_RATE = 0.001
BATCH_SIZE = 20
MAX_DECODING_DEPTH = 3
EPOCHS = 3
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [ ]:

with data_reader.data_reader(DATASET_PATH) as reader:
    dataset_pointer = reader[FILE_PATH]      
    grpahs_number = get_file_size(reader, dataset_pointer)
    shuffled_indexes = list(range(grpahs_number)) # change indexes to random_shuffle
    random.shuffle(shuffled_indexes)
    train_indexes = shuffled_indexes[:grpahs_number*7//10]
    test_indexes = shuffled_indexes[grpahs_number*7//10:]
    for max_depth in range(1, MAX_DECODING_DEPTH+1):
        for epoch in range(EPOCHS):
            # Training Loop
            correct = 0
            total = 0
            total_loss = 0
            for i in train_indexes:
                graph = dataset_pointer.node_by_id(i)
                labels = lg.get_labels(graph, max_depth)
                optimizer.zero_grad()
                output_whole = model(graph, max_depth)    
                output_whole = [j for j,_ in output_whole]
                loss = criterion(torch.stack(output_whole).squeeze(1), torch.tensor(labels))/len(labels)
                loss.backward()
                if (i + 1) % BATCH_SIZE == 0:
                    optimizer.step()
                    optimizer.zero_grad()
                total_loss += loss.item()
                predictions = torch.argmax(torch.stack(output_whole).squeeze(1), dim=1)
                correct += (predictions == torch.tensor(labels)).sum().item()
                total += len(labels)
            trainig_accuracy = correct / total if total > 0 else 0
            
            # Testing Loop
            correct = 0
            total = 0
            for i in test_indexes:
                graph = dataset_pointer.node_by_id(i)
                labels = lg.get_labels(graph, max_depth)
                with torch.no_grad():
                    output_whole = model(graph, max_depth=max_depth)

                    output_whole = [j for j,_ in output_whole]
                    predictions = torch.argmax(torch.stack(output_whole).squeeze(1), dim=1)
                    correct += (predictions == torch.tensor(labels)).sum().item()
                    total += len(labels)
            
            accuracy = correct / total if total > 0 else 0
            print(f'Max decoding depth: {max_depth}, Epoch {epoch+1}/{EPOCHS}, Training Loss: {total_loss / len(train_indexes)}, TrainingAccuracy: {trainig_accuracy* 100:.2f}%, Test Accuracy: {accuracy * 100:.2f}%')

Max decoding depth: 1, Epoch 1/3, Training Loss: 3.4073306798934935, TrainingAccuracy: 0.00%, Test Accuracy: 0.00%
Max decoding depth: 1, Epoch 2/3, Training Loss: 3.4073306798934935, TrainingAccuracy: 0.00%, Test Accuracy: 0.00%
Max decoding depth: 1, Epoch 3/3, Training Loss: 3.4073306798934935, TrainingAccuracy: 0.00%, Test Accuracy: 0.00%
Max decoding depth: 2, Epoch 1/3, Training Loss: 1.2484236359596252, TrainingAccuracy: 0.00%, Test Accuracy: 0.00%
Max decoding depth: 2, Epoch 2/3, Training Loss: 1.2484236359596252, TrainingAccuracy: 0.00%, Test Accuracy: 0.00%
Max decoding depth: 2, Epoch 3/3, Training Loss: 1.2484236359596252, TrainingAccuracy: 0.00%, Test Accuracy: 0.00%
Max decoding depth: 3, Epoch 1/3, Training Loss: 0.7662498563528061, TrainingAccuracy: 0.00%, Test Accuracy: 0.00%
Max decoding depth: 3, Epoch 2/3, Training Loss: 0.7662498563528061, TrainingAccuracy: 0.00%, Test Accuracy: 0.00%
Max decoding depth: 3, Epoch 3/3, Training Loss: 0.7662498563528061, TrainingAcc

In [43]:
sum([1 for i in labels if i[1]==3])/len(labels)

0.505826307491946

In [44]:
import pandas as pd
from collections import Counter
# Count frequencies of each tuple
frequency_table = Counter(labels)

# Convert the frequency table into a pandas DataFrame
df = pd.DataFrame(frequency_table.items(), columns=['Tuple', 'Frequency']).sort_values('Frequency', ascending=False)
df

,Tuple,Frequency
13,"(16, 3)",3790
6,"(2, 3)",3540
10,"(16, 2)",2883
11,"(9, 3)",2142
0,"(16, 1)",2073
1,"(9, 2)",1835
7,"(13, 3)",1524
5,"(2, 2)",1415
19,"(3, 3)",1107
8,"(9, 1)",828


In [45]:
with data_reader.data_reader(DATASET_PATH) as reader:
    dataset_pointer = reader[FILE_PATH]      
    grpahs_number = get_file_size(reader, dataset_pointer)
    shuffled_indexes = list(range(grpahs_number)) # change indexes to random_shuffle
    random.shuffle(shuffled_indexes)
    train_indexes = shuffled_indexes[:grpahs_number*7//10]
    test_indexes = shuffled_indexes[grpahs_number*7//10:]
    for max_depth in range(1, MAX_DECODING_DEPTH+1):
        for epoch in range(EPOCHS):
            # Training Loop
            correct = 0
            total = 0
            total_loss = 0
            for i in train_indexes:
                graph = dataset_pointer.node_by_id(i)
                labels = lg.get_labels(graph, max_depth)
                optimizer.zero_grad()
                output_whole = model(graph, max_depth=max_depth)
                loss = criterion(torch.stack(output_whole).squeeze(1), torch.tensor(labels))/len(labels)
                loss.backward()
                if (i + 1) % BATCH_SIZE == 0:
                    optimizer.step()
                    optimizer.zero_grad()
                total_loss += loss.item()
                predictions = torch.argmax(torch.stack(output_whole).squeeze(1), dim=1)
                correct += (predictions == torch.tensor(labels)).sum().item()
                total += len(labels)
            trainig_accuracy = correct / total if total > 0 else 0
            
            # Testing Loop
            correct = 0
            total = 0
            for i in test_indexes:
                graph = dataset_pointer.node_by_id(i)
                labels = lg.get_labels(graph, max_depth)
                with torch.no_grad():
                    output_whole = model(graph, max_depth=max_depth)
                    predictions = torch.argmax(torch.stack(output_whole).squeeze(1), dim=1)
                    correct += (predictions == torch.tensor(labels)).sum().item()
                    total += len(labels)
            
            accuracy = correct / total if total > 0 else 0
            print(f'Max decoding depth: {max_depth}, Epoch {epoch+1}/{EPOCHS}, Training Loss: {total_loss / len(train_indexes)}, TrainingAccuracy: {trainig_accuracy* 100:.2f}%, Test Accuracy: {accuracy * 100:.2f}%')

/tmp/ipykernel_151227/1298373791.py:43: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probabilities = F.softmax(logits)


Max decoding depth: 1, Epoch 1/3, Training Loss: 3.0268222567835505, TrainingAccuracy: 69.56%, Test Accuracy: 88.52%
Max decoding depth: 1, Epoch 2/3, Training Loss: 2.6563084640404577, TrainingAccuracy: 91.91%, Test Accuracy: 93.54%
Max decoding depth: 1, Epoch 3/3, Training Loss: 2.560238545954884, TrainingAccuracy: 94.84%, Test Accuracy: 95.68%
Max decoding depth: 2, Epoch 1/3, Training Loss: 1.2131451236901636, TrainingAccuracy: 45.07%, Test Accuracy: 44.89%
Max decoding depth: 2, Epoch 2/3, Training Loss: 1.2044079635182245, TrainingAccuracy: 45.89%, Test Accuracy: 46.92%
Max decoding depth: 2, Epoch 3/3, Training Loss: 1.1994367652754854, TrainingAccuracy: 52.89%, Test Accuracy: 59.98%
Max decoding depth: 3, Epoch 1/3, Training Loss: 0.8402869035606252, TrainingAccuracy: 38.94%, Test Accuracy: 40.82%
Max decoding depth: 3, Epoch 2/3, Training Loss: 0.8346686192707277, TrainingAccuracy: 44.21%, Test Accuracy: 47.12%
Max decoding depth: 3, Epoch 3/3, Training Loss: 0.83091205316668